# 1. Configuración

Este notebook replica el flujo del baseline de PANNs, pero insertando un bloque de **fine-tuning** más parecido al de MERT: además de una cabeza de proyección para embeddings, se permite **adaptación parcial del backbone** (últimos bloques convolucionales) para especializar el modelo al dominio EGFxSet.

Objetivos:

- Medir tiempo total de entrenamiento.
- Mantener protocolo de evaluación idéntico (Top‑1/Top‑5 por similitud coseno).
- Comparar con: (i) PANNs preentrenado, (ii) fine-tuning con cabeza congelando backbone, (iii) este experimento con backbone parcialmente entrenable.

## 1. Imports centralizados

Importa librerías base y define utilidades comunes.

In [1]:
import sys
import os
from pathlib import Path

import numpy as np
import pandas as pd

import torch
import torchaudio
import soundfile as sf

import re

from tqdm import tqdm
import transformers
from transformers import AutoModel

/opt/anaconda3/envs/mert310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_ROOT = Path("/Users/dtenreiro/Documents/TFM/EGFxSet")
OUT_DIR   = Path("./outputs_01_panns_base_egfxset")

OUT_DIR.mkdir(parents=True, exist_ok=True)

print("DATA_ROOT exists:", DATA_ROOT.exists(), "->", DATA_ROOT)
print("OUT_DIR exists:", OUT_DIR.exists(), "->", OUT_DIR)

DATA_ROOT exists: True -> /Users/dtenreiro/Documents/TFM/EGFxSet
OUT_DIR exists: True -> outputs_01_panns_base_egfxset


## 2. Semilla y carpeta de salida

Fija semilla (reproducibilidad) y define el directorio de salida de este experimento (02).

In [3]:
import os, random
from pathlib import Path
import numpy as np
import torch

OUT_DIR_03 = Path("./outputs_03_panns_finetune_egfxset_unfreeze")
OUT_DIR_03.mkdir(parents=True, exist_ok=True)

SEED = 42
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(SEED)
print("OUT_DIR_03:", OUT_DIR_03.resolve())

OUT_DIR_03: /Users/dtenreiro/Documents/TFM/panns_inference/outputs_03_panns_finetune_egfxset_unfreeze


## 3. Dispositivo

Selecciona `mps/cuda/cpu` igual que en el baseline.

In [4]:
device = (
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)
device

if device == "cuda":
    print("CUDA device:", torch.cuda.get_device_name(0))
elif device == "mps":
    print("Apple Silicon MPS enabled")
else:
    print("CPU only")

Apple Silicon MPS enabled


## 4. Sincronización para benchmarks

Define `_sync()` para que las medidas de tiempo sean correctas en GPU/MPS.

In [5]:
import time
import numpy as np
import torch

def _sync():
    # Para que el cronómetro mida de verdad en GPU/MPS
    if device == "cuda" and torch.cuda.is_available():
        torch.cuda.synchronize()
    elif device == "mps" and hasattr(torch, "mps"):
        try:
            torch.mps.synchronize()
        except Exception:
            pass

def _summ(times_s, label=""):
    times = np.array(times_s, dtype=np.float64)
    return {
        "label": label,
        "n": int(times.size),
        "mean_ms": float(times.mean() * 1000),
        "p50_ms": float(np.percentile(times, 50) * 1000),
        "p95_ms": float(np.percentile(times, 95) * 1000),
        "min_ms": float(times.min() * 1000),
        "max_ms": float(times.max() * 1000),
        "runs_per_s": float(1.0 / times.mean()),
    }

# 2. Carga del índice del dataset

Reutiliza el `egfxset_index.csv` generado por el baseline de PANNs para mantener el **mismo orden** y metadatos.

In [6]:
import pandas as pd
from pathlib import Path

INDEX_CSV_PATH = Path("./outputs_01_panns_base_egfxset/egfxset_index.csv")
assert INDEX_CSV_PATH.exists(), f"No encuentro {INDEX_CSV_PATH}. Ejecuta antes el notebook 01_PANNs_base_egfx."

df = pd.read_csv(INDEX_CSV_PATH)
print("Index:", df.shape)
df.head()

Index: (8947, 7)


,path,tone,pickup,tone_pickup,string,fret,midi_pitch
0,/Users/dtenreiro/Documents/TFM/EGFxSet/BluesDr...,BluesDriver,Bridge,BluesDriver__Bridge,1,0,64
1,/Users/dtenreiro/Documents/TFM/EGFxSet/BluesDr...,BluesDriver,Bridge,BluesDriver__Bridge,1,1,65
2,/Users/dtenreiro/Documents/TFM/EGFxSet/BluesDr...,BluesDriver,Bridge,BluesDriver__Bridge,1,10,74
3,/Users/dtenreiro/Documents/TFM/EGFxSet/BluesDr...,BluesDriver,Bridge,BluesDriver__Bridge,1,11,75
4,/Users/dtenreiro/Documents/TFM/EGFxSet/BluesDr...,BluesDriver,Bridge,BluesDriver__Bridge,1,12,76


## 1. Configuración del objetivo de fine-tuning

La tarea de fine-tuning será **clasificar `tone_pickup`** (65 clases), como en el fine-tuning de MERT.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

LABEL_COL = "tone_pickup"

df["row_id"] = np.arange(len(df), dtype=int)

le = LabelEncoder()
df["label_id"] = le.fit_transform(df[LABEL_COL].astype(str))
N_CLASSES = len(le.classes_)
print("N_CLASSES:", N_CLASSES)

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=SEED,
    stratify=df["label_id"]
)

train_df = train_df.reset_index(drop=True)
val_df   = val_df.reset_index(drop=True)

train_idx = train_df["row_id"].to_numpy()
val_idx   = val_df["row_id"].to_numpy()

print("Train:", train_df.shape, "| Val:", val_df.shape)
print("train_idx:", train_idx.shape, "| val_idx:", val_idx.shape)

N_CLASSES: 65
Train: (7157, 9) | Val: (1790, 9)
train_idx: (7157,) | val_idx: (1790,)


# 3. PANNs: carga y fine-tuning

Cargamos PANNs CNN14 preentrenado (igual que en el baseline) y definimos una cabeza de clasificación para `tone_pickup`.

## 1. Parámetros de audio y utilidades de pooling

Se mantiene el mismo preprocesado (32 kHz, recorte/padding a 5 s).

In [8]:
MAX_SECONDS = 5.0
TARGET_SR = 32000

def load_audio_panns(path: str | Path):
    """Carga wav -> mono -> resample a 32kHz -> recorta a 5s"""
    wav, sr = sf.read(str(path))

    if wav.ndim == 2:
        wav = wav.mean(axis=1)

    wav = torch.from_numpy(wav).float()

    if sr != TARGET_SR:
        wav = torchaudio.functional.resample(wav, sr, TARGET_SR)
        sr = TARGET_SR

    wav = wav[: int(sr * MAX_SECONDS)]
    return wav, sr

def temporal_pool(x: torch.Tensor, mode: str) -> torch.Tensor:
    """
    x: (T, D) ó (B, T, D). Devuelve (D) ó (B, D)
    """
    if x.dim() == 2:
        x0 = x
        if mode == "mean":
            return x0.mean(dim=0)
        elif mode == "max":
            return x0.max(dim=0).values
        elif mode == "stats":
            mu = x0.mean(dim=0)
            sd = x0.std(dim=0, unbiased=False)
            return torch.cat([mu, sd], dim=0)
        else:
            raise ValueError(f"Unknown mode: {mode}")

    if x.dim() == 3:
        if mode == "mean":
            return x.mean(dim=1)
        elif mode == "max":
            return x.max(dim=1).values
        elif mode == "stats":
            mu = x.mean(dim=1)
            sd = x.std(dim=1, unbiased=False)
            return torch.cat([mu, sd], dim=2)
        else:
            raise ValueError(f"Unknown mode: {mode}")

    raise ValueError(f"Unexpected tensor rank: {x.dim()}")

## 2. Dataset + DataLoaders

Construye un `Dataset` y un `collate_fn` que:
- Carga WAV
- Convierte a mono
- Re-muestrea a 32 kHz
- Recorta o **rellena con ceros** hasta 5 s

Esto permite hacer batches con tensores de longitud fija.

In [9]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

PATH_COL = "path"
LABEL_ID_COL = "label_id"

MAX_LEN = int(TARGET_SR * MAX_SECONDS)

def load_audio_fixed(path: str | Path):
    wav, sr = load_audio_panns(path)

    if wav.numel() < MAX_LEN:
        pad = MAX_LEN - wav.numel()
        wav = torch.nn.functional.pad(wav, (0, pad))
    else:
        wav = wav[:MAX_LEN]
    return wav

class EGFxSetToneDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        wav = load_audio_fixed(row[PATH_COL])
        y = int(row[LABEL_ID_COL])
        return wav, y

def collate_fn(batch):
    wavs, ys = zip(*batch)
    x = torch.stack([w.float() for w in wavs], dim=0)  # (B, T)
    y = torch.tensor(ys, dtype=torch.long)
    return x, y

BATCH_SIZE = 16
train_loader = DataLoader(EGFxSetToneDataset(train_df), batch_size=BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=collate_fn)
val_loader   = DataLoader(EGFxSetToneDataset(val_df),   batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=collate_fn)

## 3. Cargar PANNs CNN14 + checkpoint

Reutiliza el mismo bloque del baseline para localizar y cargar el `.pth`.

In [10]:
from pathlib import Path
import urllib.request

OUT = Path("pretrained_models")
OUT.mkdir(parents=True, exist_ok=True)

url = "https://zenodo.org/record/3987831/files/Cnn14_mAP%3D0.431.pth?download=1"
ckpt = OUT / "Cnn14_mAP=0.431.pth"

if not ckpt.exists():
    urllib.request.urlretrieve(url, ckpt)

PANNS_CNN14_CKPT = str(ckpt)
print("Checkpoint:", PANNS_CNN14_CKPT)

Checkpoint: pretrained_models/Cnn14_mAP=0.431.pth


In [11]:
from pathlib import Path
import os
import inspect
import torch
from panns_inference.models import Cnn14

def find_cnn14_ckpt() -> Path:

    env = os.environ.get("PANNS_CNN14_CKPT", "").strip()
    if env:
        p = Path(env).expanduser()
        if p.exists() and p.is_file():
            return p
        raise FileNotFoundError(f"PANNS_CNN14_CKPT not found: {p}")

    pkg_root = Path(inspect.getfile(Cnn14)).resolve().parent
    candidates = []
    for rel in [
        "../pretrained_models",
        "../pretrained",
        "../data",
        "../../pretrained_models",
        "../../pretrained",
    ]:
        d = (pkg_root / rel).resolve()
        if d.exists() and d.is_dir():
            candidates += list(d.rglob("*.pth"))

    candidates = sorted(candidates, key=lambda p: (("cnn14" not in p.name.lower()), len(p.name)))
    if candidates:
        return candidates[0]

    raise FileNotFoundError(
        "No se ha encontrado ningún checkpoint .pth de PANNs CNN14.\n"
    )

CKPT = find_cnn14_ckpt()
print("Using checkpoint:", CKPT)

panns_model = Cnn14(
    sample_rate=TARGET_SR,
    window_size=1024,
    hop_size=320,
    mel_bins=64,
    fmin=50,
    fmax=14000,
    classes_num=527
)

ckpt = torch.load(CKPT, map_location=device)
state = ckpt["model"] if isinstance(ckpt, dict) and "model" in ckpt else ckpt
panns_model.load_state_dict(state, strict=True)

panns_model.to(device).eval()
print("Loaded PANNs:", panns_model.__class__.__name__, "on", device)

Using checkpoint: /Users/dtenreiro/Documents/TFM/panns_inference/pretrained_models/Cnn14_mAP=0.431.pth
Loaded PANNs: Cnn14 on mps


## 4. Modelo para fine-tuning

Crea un wrapper que:
- Obtiene el embedding global de PANNs (`out['embedding']`).
- Aplica una capa `Linear` para predecir `tone_pickup`.

Así podemos entrenar una cabeza ligera y, opcionalmente, descongelar una parte final del encoder.

## Fine-tuning con *projection head* + adaptación parcial del backbone

**Objetivo**: mejorar la calidad del espacio de embeddings para *retrieval* (Top‑1/Top‑5) manteniendo el pooling interno de PANNs, pero permitiendo una **adaptación controlada** de las **últimas capas** del encoder.

**Qué cambia respecto al notebook anterior**:
- Además de la proyección `2048 → 256`, se **descongelan** los últimos bloques convolucionales para que el modelo pueda especializarse al dominio EGFxSet (estrategia más cercana al fine‑tuning parcial aplicado en MERT).
- El resto del protocolo (split, ventanas, evaluación por similitud coseno) se mantiene idéntico para comparar de forma directa.


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PANNsForTonePickup(nn.Module):
    """Wrapper de fine-tuning para PANNs.

    - backbone: PANNs (CNN14) preentrenado
    - proj: proyección entrenable para compactar el embedding (mejor retrieval)
    - classifier: capa final de clasificación

    Nota: el pooling temporal interno de PANNs NO se modifica; aquí solo se adapta el embedding final.
    """
    def __init__(self, panns_backbone: nn.Module, emb_dim: int, proj_dim: int, n_classes: int):
        super().__init__()
        self.panns = panns_backbone
        self.proj = nn.Linear(emb_dim, proj_dim)
        self.classifier = nn.Linear(proj_dim, n_classes)

    def forward(self, x):
        out = self.panns(x, None)
        emb_raw = out["embedding"]               # (B, emb_dim)
        emb = self.proj(emb_raw)                # (B, proj_dim)
        emb = F.normalize(emb, p=2, dim=-1)
        logits = self.classifier(emb)
        return logits, emb, emb_raw

panns_model.eval()
with torch.no_grad():
    x0, _ = next(iter(train_loader))
    x0 = x0.to(device)
    out0 = panns_model(x0, None)
    emb_dim = int(out0["embedding"].shape[-1])

PROJ_DIM = 256
ft_model = PANNsForTonePickup(panns_model, emb_dim=emb_dim, proj_dim=PROJ_DIM, n_classes=N_CLASSES).to(device)
print("emb_dim:", emb_dim, "| proj_dim:", PROJ_DIM)

emb_dim: 2048 | proj_dim: 256


### 5. Fine-tuning parcial de PANNs (CNN14)

En este experimento se parte del modelo CNN14 preentrenado y se realiza un fine-tuning **parcial y controlado**, entrenando:
- La **cabeza de proyección** (2048→256) y la **capa de clasificación**.
- Un subconjunto reducido del encoder (los **últimos bloques convolucionales**), para adaptar las representaciones al dominio específico de guitarra eléctrica sin alterar por completo el conocimiento general aprendido en AudioSet.


## Política de congelación

Para maximizar la comparabilidad con el fine‑tuning de MERT y evitar inestabilidad:
- **Congelamos** la mayor parte del backbone PANNs.
- Entrenamos siempre:
  - `proj` (proyección 2048→256)
  - `classifier`
- Además, **descongelamos** únicamente los **últimos bloques** del encoder (por defecto `conv_block5` y `conv_block6`), manteniendo el resto fijo.

Esta estrategia busca un compromiso entre **adaptación al dominio** y **riesgo de sobreajuste**.


In [13]:
import torch.nn as nn

backbone = ft_model.panns

for p in backbone.parameters():
    p.requires_grad = False

for p in ft_model.proj.parameters():
    p.requires_grad = True
for p in ft_model.classifier.parameters():
    p.requires_grad = True

UNFREEZE_BLOCKS = ["conv_block5", "conv_block6"]
UNFREEZE_BN_ONLY = False

def _set_trainable(module: nn.Module, trainable: bool):
    for p in module.parameters():
        p.requires_grad = trainable

for name in UNFREEZE_BLOCKS:
    if not hasattr(backbone, name):
        raise AttributeError(f"Backbone no tiene atributo '{name}'. Revisa nombres disponibles en backbone.")
    block = getattr(backbone, name)
    if UNFREEZE_BN_ONLY:

        for m in block.modules():
            if isinstance(m, nn.BatchNorm2d):
                for p in m.parameters():
                    p.requires_grad = True
    else:
        _set_trainable(block, True)

print("UNFREEZE_BLOCKS:", UNFREEZE_BLOCKS, "| UNFREEZE_BN_ONLY:", UNFREEZE_BN_ONLY)

UNFREEZE_BLOCKS: ['conv_block5', 'conv_block6'] | UNFREEZE_BN_ONLY: False


In [14]:
def count_params(module):
    return sum(p.numel() for p in module.parameters())

def count_trainable_params(module):
    return sum(p.numel() for p in module.parameters() if p.requires_grad)

print("Trainable total:", sum(p.numel() for p in ft_model.parameters() if p.requires_grad))

print("Trainable classifier:", count_trainable_params(ft_model.classifier), "/", count_params(ft_model.classifier))

print("Trainable conv_block6:", count_trainable_params(ft_model.panns.conv_block6), "/", count_params(ft_model.panns.conv_block6))

others = []
for name, p in ft_model.named_parameters():
    if p.requires_grad and not (name.startswith("panns.conv_block6.") or name.startswith("classifier.")):
        others.append(name)
print("Otros entrenables fuera de conv_block6+classifier:", others[:20], " (n=", len(others), ")")

Trainable total: 71332417
Trainable classifier: 16705 / 16705
Trainable conv_block6: 56631296 / 56631296
Otros entrenables fuera de conv_block6+classifier: ['panns.conv_block5.conv1.weight', 'panns.conv_block5.conv2.weight', 'panns.conv_block5.bn1.weight', 'panns.conv_block5.bn1.bias', 'panns.conv_block5.bn2.weight', 'panns.conv_block5.bn2.bias', 'proj.weight', 'proj.bias']  (n= 8 )


## 6. Loop de entrenamiento

Entrena con `CrossEntropyLoss`, early stopping por `val_loss` y guarda el mejor estado.

También mide el **tiempo total de entrenamiento** (wall-clock).

## Control explícito de `train()` / `eval()`

Aunque el backbone esté congelado (sin gradiente), **BatchNorm/Dropout** pueden comportarse distinto según `train()`/`eval()`.
Aquí dejamos el backbone global en `eval()` para evitar deriva de estadísticos, y si `UNFREEZE_BN_BLOCK6=True` activamos solo `conv_block6` en `train()` para permitir adaptación de BN en ese bloque.


In [15]:
backbone = ft_model.panns

backbone.eval()
ft_model.proj.train()
ft_model.classifier.train()

if 'UNFREEZE_BN_BLOCK6' in globals() and UNFREEZE_BN_BLOCK6:
    backbone.conv_block6.train()

## Entrenamiento

Entrenamos únicamente los parámetros marcados como entrenables (proyección + classifier + BN opcional).
En cada época fijamos el modo de trabajo:
- backbone en `eval()`
- `proj` y `classifier` en `train()`
- (opcional) `conv_block6` en `train()` si queremos adaptar BN.

Además, como el `forward` ahora devuelve `(logits, emb_proj, emb_raw)`, usamos `logits` para la pérdida y mantenemos `emb_proj` para extracción posterior.


In [16]:
from tqdm import tqdm
import time
import copy

criterion = nn.CrossEntropyLoss()

LR_HEAD = 1e-4
LR_BACKBONE = 1e-5
WEIGHT_DECAY = 1e-4
EPOCHS_MAX = 30
PATIENCE = 6
GRAD_CLIP = 1.0
MIN_DELTA = 1e-3

best_val = float("inf")
best_state = None
best_epoch = -1
no_improve = 0

head_params = [p for p in list(ft_model.proj.parameters()) + list(ft_model.classifier.parameters()) if p.requires_grad]
backbone_params = [p for p in ft_model.panns.parameters() if p.requires_grad]

print(f"Parámetros entrenables (head): {sum(p.numel() for p in head_params):,}")
print(f"Parámetros entrenables (backbone): {sum(p.numel() for p in backbone_params):,}")
print(f"Parámetros entrenables (total): {sum(p.numel() for p in ft_model.parameters() if p.requires_grad):,} / {sum(p.numel() for p in ft_model.parameters()):,}")

param_groups = [
    {"params": head_params, "lr": LR_HEAD, "weight_decay": WEIGHT_DECAY},
]
if len(backbone_params) > 0:
    param_groups.append({"params": backbone_params, "lr": LR_BACKBONE, "weight_decay": WEIGHT_DECAY})

optimizer = torch.optim.AdamW(param_groups)

history = []
t0 = time.time()

def _set_backbone_train_mode():

    ft_model.panns.eval()
    for name in UNFREEZE_BLOCKS:
        block = getattr(ft_model.panns, name)
        block.train()

for epoch in range(1, EPOCHS_MAX + 1):

    ft_model.train()
    _set_backbone_train_mode()
    ft_model.proj.train()
    ft_model.classifier.train()

    tr_loss = 0.0
    tr_correct = 0
    tr_n = 0

    for x, y in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS_MAX} [train]"):
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad(set_to_none=True)
        logits, emb, emb_raw = ft_model(x)
        loss = criterion(logits, y)
        loss.backward()

        if GRAD_CLIP is not None:
            torch.nn.utils.clip_grad_norm_(ft_model.parameters(), GRAD_CLIP)

        optimizer.step()

        tr_loss += float(loss.item()) * x.size(0)
        tr_correct += int((logits.argmax(dim=1) == y).sum().item())
        tr_n += x.size(0)

    tr_loss /= max(1, tr_n)
    tr_acc = tr_correct / max(1, tr_n)


    ft_model.eval()
    va_loss = 0.0
    va_correct = 0
    va_n = 0

    with torch.no_grad():
        for x, y in tqdm(val_loader, desc=f"Epoch {epoch}/{EPOCHS_MAX} [val]"):
            x = x.to(device)
            y = y.to(device)
            logits, emb, emb_raw = ft_model(x)
            loss = criterion(logits, y)

            va_loss += float(loss.item()) * x.size(0)
            va_correct += int((logits.argmax(dim=1) == y).sum().item())
            va_n += x.size(0)

    va_loss /= max(1, va_n)
    va_acc = va_correct / max(1, va_n)

    history.append({"epoch": epoch, "train_loss": tr_loss, "train_acc": tr_acc, "val_loss": va_loss, "val_acc": va_acc})
    print(f"Epoch {epoch}: train_loss={tr_loss:.4f} train_acc={tr_acc:.4f} | val_loss={va_loss:.4f} val_acc={va_acc:.4f}")

    if va_loss < best_val - MIN_DELTA:
        best_val = va_loss
        best_epoch = epoch
        best_state = copy.deepcopy(ft_model.state_dict())
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= PATIENCE:
            print(f"Early stopping at epoch {epoch} (best epoch={best_epoch}, best val_loss={best_val:.4f})")
            break

train_time_s = time.time() - t0
print(f"Training time: {train_time_s/60:.2f} min ({train_time_s:.1f} s)")

Parámetros entrenables (head): 541,249
Parámetros entrenables (backbone): 70,791,168
Parámetros entrenables (total): 71,332,417 / 82,378,320


Epoch 1/30 [val]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 112/112 [00:10<00:00, 10.22it/s]


Epoch 1: train_loss=4.0166 train_acc=0.2138 | val_loss=3.8376 val_acc=0.4196


Epoch 2/30 [val]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 112/112 [00:08<00:00, 12.68it/s]


Epoch 2: train_loss=3.7157 train_acc=0.4802 | val_loss=3.5703 val_acc=0.5514


...

Epoch 28: train_loss=0.3496 train_acc=0.9803 | val_loss=0.3815 val_acc=0.9458


Epoch 29/30 [val]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 112/112 [00:09<00:00, 11.68it/s]


Epoch 29: train_loss=0.3083 train_acc=0.9849 | val_loss=0.3509 val_acc=0.9464


Epoch 30/30 [val]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 112/112 [00:09<00:00, 11.30it/s]

Epoch 30: train_loss=0.2730 train_acc=0.9866 | val_loss=0.3140 val_acc=0.9531
Training time: 35.30 min (2118.2 s)


In [17]:
import pandas as pd
import torch

HIST_CSV = OUT_DIR_03 / "train_history.csv"
pd.DataFrame(history).to_csv(HIST_CSV, index=False)
print("Saved:", HIST_CSV.resolve())

BEST_PT = OUT_DIR_03 / "best.pt"
LAST_PT = OUT_DIR_03 / "last.pt"

if best_state is not None:
    ft_model.load_state_dict(best_state)
    torch.save({
        "state_dict": best_state,
        "best_epoch": best_epoch,
        "best_val_loss": best_val,
        "unfreeze_blocks": UNFREEZE_BLOCKS,
        "unfreeze_bn_only": UNFREEZE_BN_ONLY,
        "lr_head": LR_HEAD,
        "lr_backbone": LR_BACKBONE,
    }, BEST_PT)
    print("Saved best:", BEST_PT.resolve())
else:
    print("WARNING: best_state es None (no se guardó best.pt)")

torch.save({
    "state_dict": ft_model.state_dict(),
    "epoch_end": history[-1]["epoch"] if len(history) else None,
    "unfreeze_blocks": UNFREEZE_BLOCKS,
    "unfreeze_bn_only": UNFREEZE_BN_ONLY,
}, LAST_PT)
print("Saved last:", LAST_PT.resolve())

Saved: /Users/dtenreiro/Documents/TFM/panns_inference/outputs_03_panns_finetune_egfxset_unfreeze/train_history.csv
Saved best: /Users/dtenreiro/Documents/TFM/panns_inference/outputs_03_panns_finetune_egfxset_unfreeze/best.pt
Saved last: /Users/dtenreiro/Documents/TFM/panns_inference/outputs_03_panns_finetune_egfxset_unfreeze/last.pt
